In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_37289/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/13 10:30:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/13 10:30:10 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
import pandas as pd

# Sample data for the Salary table
salary_data = {
    'id': [1, 2, 3, 4, 5, 6],
    'employee_id': [1, 2, 3, 1, 2, 3],
    'amount': [9000, 6000, 10000, 7000, 6000, 8000],
    'pay_date': ['2017-03-31', '2017-03-31', '2017-03-31', '2017-02-28', '2017-02-28', '2017-02-28']
}

# Sample data for the Employee table
employee_data = {
    'employee_id': [1, 2, 3],
    'department_id': [1, 2, 2]
}

# Creating pandas DataFrames for Salary and Employee tables
salary_df = pd.DataFrame(salary_data)
employee_df = pd.DataFrame(employee_data)

# Displaying the Salary and Employee DataFrames
print("Salary table:")
print(salary_df)
print("\nEmployee table:")
print(employee_df)



df_person = spark.createDataFrame(salary_df)
df_person.createOrReplaceTempView("Salary")

df_person = spark.createDataFrame(employee_df)
df_person.createOrReplaceTempView("Employee")



Salary table:
   id  employee_id  amount    pay_date
0   1            1    9000  2017-03-31
1   2            2    6000  2017-03-31
2   3            3   10000  2017-03-31
3   4            1    7000  2017-02-28
4   5            2    6000  2017-02-28
5   6            3    8000  2017-02-28

Employee table:
   employee_id  department_id
0            1              1
1            2              2
2            3              2


In [26]:
query = """
    select month as pay_month, department_id, comparison from (select
        distinct month, department_id, deparment_salary_per_month, company_salary_per_month,
        case
            when deparment_salary_per_month = company_salary_per_month then 'same'
            when deparment_salary_per_month > company_salary_per_month then 'higher'
            else 'lower'
        end as comparison 
    from (
        select
            s.employee_id,
            s.amount,
            s.pay_date,
            e.department_id,
            avg(amount) over (partition by year(s.pay_date), month(s.pay_date)) as company_salary_per_month,
            avg(amount) over (partition by year(s.pay_date), month(s.pay_date), e.department_id) as deparment_salary_per_month,
            CONCAT(YEAR(pay_date), '-', LPAD(MONTH(pay_date), 2, '0')) as month
        from Salary s left join Employee e on s.employee_id = e.employee_id
        order by s.pay_date, e.department_id
    )) order by department_id, pay_month
"""

In [27]:
spark.sql(query).show()

+---------+-------------+----------+
|pay_month|department_id|comparison|
+---------+-------------+----------+
|  2017-02|            1|      same|
|  2017-03|            1|    higher|
|  2017-02|            2|      same|
|  2017-03|            2|     lower|
+---------+-------------+----------+

